#Libraries

In [21]:
# import additional stuff needed later on:
import json
import random


In [22]:
# import Paillier library
!pip install phe
import phe
from phe import paillier

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
pubkey, privkey = paillier.generate_paillier_keypair(n_length=1024)

#Ruf

In [24]:
# p = privkey.p
# q = privkey.q

# g = pubkey.g
# n = pubkey.n

In [25]:
m1 = 3.14
m2 = 10.02

M1 = pubkey.encrypt(m1)
M2 = pubkey.encrypt(m2)

In [26]:
M3 = M1 + M2

# Create some constants
n1 = 2
n2 = 0.125

M4 = M1 * n1

M5 = M1 + n2

# Multiplication of two ciphertexts are not supported
# M6 = M1 * M2

In [27]:
print(M3.ciphertext())
print(M4.ciphertext())
print(M5.ciphertext())

10370575607910384970553999608572355365013253139409160933381978873108249424051568623480064705326251458414099984816348779384260130904636995188838929767056975779770314412715866773086864944700499289412651814288616700910903916306738835999340663604711918543333036569383884187488967304272510096149983322077536012240163440002452130265845444094535416161276009770709297741498895142238203594471096796105119972474450068385690504994942136471272451005141174665564301770809775840338526122081773377549638095168008687722712644527937939393763699079777320183777187590351476056942059341732605968405648563636467563011674436739763377432464
1090163498023873994761470502811676823499925001686980002308149477218327568610521899981759389439047972246954633059364003391874727688203431552913103723270248392700438101921896434939304748280110704057391347620806406059744867580722151655738712222457718431105322812076528831795844972211768888023821001823479220102069778979702071252454067442637189458419566410002278177578435974927381490807

In [28]:
m3 = privkey.decrypt(M3)
m4 = privkey.decrypt(M4)
m5 = privkey.decrypt(M5)

print(m3)
print(m4)
print(m5)

print("Original values:")
print(m1 + m2)
print(n1 * m1)
print(m1 + n2)

13.16
6.28
3.265
Original values:
13.16
6.28
3.265


In [29]:
print("Public Key:")
print(pubkey.n)
print(len(str(pubkey.n)))
print(pubkey.g)
print(pubkey.g - pubkey.n)
if pubkey.n%2:
    print("N is odd.")
print()

print("Private Key:")
print(privkey.p)
print(len(str(privkey.p)))
print(privkey.q)
print(len(str(privkey.q)))

Public Key:
126501615513353922317699021840088376737423467391657129805210170487346722672698309258966017716913468255379644345498200259504401019004016820084013017245972519822864923404388879589946148072009453360485366774131321260299985310757880275475906287344998044869237304458368321884670207862982964487699138431358907416391
309
126501615513353922317699021840088376737423467391657129805210170487346722672698309258966017716913468255379644345498200259504401019004016820084013017245972519822864923404388879589946148072009453360485366774131321260299985310757880275475906287344998044869237304458368321884670207862982964487699138431358907416392
1
N is odd.

Private Key:
10691915782657578257929633835902946453017032931397259316140018917491330018075011106391128926982537433930363527819030132379370494119300790403028627649474303
155
11831520008653746801627106640779548027373059254407950199101409979435558116592468860801663721981932722307499381831885787986484572003949576432621541978503097
155


In [30]:
for i in range(4, 0, -1):
    print(i)

4
3
2
1


#SBD from MSB to LSB

In [31]:
def SBD_MSB(E_X, m):
    Value_E_X = E_X
    l = []
# Stage 1:

    # Bob:
    # r = random.randint(1, pubkey.n - (2 ** m - 1) - 1)
    r = random.randint(1, 100)
    E_y = E_X + r

    # Alice:
    lam = 0
    y = privkey.decrypt(E_y)
    if(y % 2 == 0):
        lam = 0
    else:
        lam = 1
    lam_stage1 = lam
    E_lam = pubkey.encrypt(lam)

    # Bob:
    if(r%2 == 0):
        E_X = E_X + 1
        E_X = E_X - E_lam
    else:
        E_X = E_X + E_lam
    # print("Stage 1 complete")
    
    
# Stage 2:
    
    for i in range(m - 1, 0, -1):
        # Bob:
        # print(i, "1")

        r1 = random.randint(1, 10000)
        r2 = random.randint(1, 10000)
        r3 = random.randint(1, r2 - 1)
        y1 = 1<<(i)

        E_T1 = pubkey.encrypt(0)
        E_T2 = pubkey.encrypt(0)

        E_T1 = E_X + r1
        E_T1 = E_T1 * r2
        E_T1 = E_T1 - r3

        E_T2 = y1 + r1
        E_T2 = E_T2 * r2

        B = random.randint(0, 1)
        Z1, Z2 = pubkey.encrypt(0), pubkey.encrypt(0)
        if B == 1:
            Z1 = pubkey.encrypt(0) + E_T1
            Z2 = pubkey.encrypt(0) + E_T2
        else:
            Z1 = pubkey.encrypt(0) + E_T2
            Z2 = pubkey.encrypt(0) + E_T1
        
        # print(i, '2')

        # Alice:

        z1 = privkey.decrypt(Z1)
        # print(i, '21')
        z2 = privkey.decrypt(Z2)
        # print(i, '22')

        if(z1 > z2):
            lam = 1
        else:
            lam = 0
        # print(i, '23')
        E_lam = pubkey.encrypt(lam)
        

        # print(i, '3')

        # Bob:
        E_X_bit = 0
        if B == 1:
            E_X_bit = E_lam
        else:
            E_X_bit = pubkey.encrypt(1) - E_lam
        l.append(E_X_bit)
        E_X = E_X - E_X_bit * y1
    

    # Bob:
    if(r%2 == 0):
        E_X = E_X - 1
        E_X = E_X + lam_stage1
    else:
        E_X = E_X - lam_stage1
    
    l.append(E_X)
    # print("Stage 2 complete")




# Stage 3:

    # Bob:
    sum = pubkey.encrypt(0)
    t = 2**(m - 1)
    for i in range(0, m):
        sum = sum + l[i] * t
        t /= 2
    V = sum - Value_E_X
    rr = random.randint(1, 1000)
    W = V * rr

    # Alice:
    D_W = privkey.decrypt(W)
    if D_W:
        # print("Stage 3 complete")
        l = SBD_MSB(pubkey, privkey, Value_E_X, m)
        return l
    else: 
        # print("Stage 3 complete")
        return l
    

In [ ]:
X = 59
T = X
l2 = []
while T > 0:
    l2.append(T % 2)
    T = int(T/2)
l2.reverse()

print("Actual value of X in bit representatioin:")
for i in l2:
    print(i, end = "")
print()

print("Value that we found from SBD algorithm:")
E_X = pubkey.encrypt(X)
l = SBD_MSB(E_X, X.bit_length())
for i in l:
    print(privkey.decrypt(i), end = "")

Actual value of X in bit representatioin:
111011
Value that we found from SBD algorithm:
111011

In [ ]:
def SBD_MSB_2(E_X, m):
    Value_E_X = E_X
    l = []
# Stage 1:

    # Bob:
    # r = random.randint(1, pubkey.n - (2 ** m - 1) - 1)
    r = random.randint(1, 100)
    E_y = E_X + r

    # Alice:
    lam = 0
    y = privkey.decrypt(E_y)
    if(y % 2 == 0):
        lam = 0
    else:
        lam = 1
    lam_stage1 = lam
    E_lam = pubkey.encrypt(lam)

    # Bob:
    if(r%2 == 0):
        E_X = E_X + 1
        E_X = E_X - E_lam
    else:
        E_X = E_X + E_lam
    # print("Stage 1 complete")
    
    
# Stage 2:
    
    for i in range(m - 1, m - 2, -1):
        # Bob:
        # print(i, "1")

        r1 = random.randint(1, 100)
        r2 = random.randint(1, 100)
        r3 = random.randint(1, r2 - 1)
        y1 = 1<<(i)

        E_T1 = pubkey.encrypt(0)
        E_T2 = pubkey.encrypt(0)

        E_T1 = E_X + r1
        E_T1 = E_T1 * r2
        E_T1 = E_T1 - r3

        E_T2 = y1 + r1
        E_T2 = E_T2 * r2

        B = random.randint(0, 1)
        Z1, Z2 = pubkey.encrypt(0), pubkey.encrypt(0)
        if B == 1:
            Z1 = pubkey.encrypt(0) + E_T1
            Z2 = pubkey.encrypt(0) + E_T2
        else:
            Z1 = pubkey.encrypt(0) + E_T2
            Z2 = pubkey.encrypt(0) + E_T1
        
        # print(i, '2')

        # Alice:

        z1 = privkey.decrypt(Z1)
        # print(i, '21')
        z2 = privkey.decrypt(Z2)
        # print(i, '22')

        if(z1 > z2):
            lam = 1
        else:
            lam = 0
        # print(i, '23')
        E_lam = pubkey.encrypt(lam)
        

        # print(i, '3')

        # Bob:
        E_X_bit = 0
        if B == 1:
            E_X_bit = E_lam
        else:
            E_X_bit = pubkey.encrypt(1) - E_lam
        l.append(E_X_bit)
        E_X = E_X - E_X_bit * y1
    
    return l
    # Bob:
    # if(r%2 == 0):
    #     E_X = E_X - 1
    #     E_X = E_X + lam_stage1
    # else:
    #     E_X = E_X - lam_stage1
    
    # l.append(E_X)
    # print("Stage 2 complete")




# Stage 3:

    # Bob:
    # sum = pubkey.encrypt(0)
    # t = 2**(m - 1)
    # for i in range(0, m):
    #     sum = sum + l[i] * t
    #     t /= 2
    # V = sum - Value_E_X
    # rr = random.randint(1, 1000)
    # W = V * rr

    # # Alice:
    # D_W = privkey.decrypt(W)
    # if D_W:
    #     # print("Stage 3 complete")
    #     l = SBD_MSB(pubkey, privkey, Value_E_X, m)
    #     return l
    # else: 
    #     # print("Stage 3 complete")
    #     return l
    

In [ ]:
X = 7
print(X.bit_length())

E_X = pubkey.encrypt(X)
l = SBD_MSB_2(E_X, X.bit_length())
for i in l:
    print(privkey.decrypt(i), end = "")

3
1

In [ ]:
# def SBD_MSB_iterations(m, n_length, iterations):
#     pubkey, privkey = paillier.generate_paillier_keypair(n_length=1024)

#     for i in range(iterations):
#         X = random.randint(1, 2**m - 1)
#         l = SBD_MSB(pubkey, privkey, pubkey.encrypt(X), m)
#         print(X)
#         print(l)
#         values = [privkey.decrypt(j) for j in l]
#         print()


In [ ]:
print(1<<2)

4


In [ ]:
abstract
applications of SBD